In [ ]:
import pandas as pd
import re

# Load the dataset (reusing loading logic for robustness)
try:
    df = pd.read_csv('final_food_delivery_dataset.csv')
    df['order_date'] = pd.to_datetime(df['order_date'], dayfirst=True)
except:
    # Fallback to regeneration
    orders_df = pd.read_csv('orders.csv')
    users_df = pd.read_json('users.json')
    restaurant_data = []
    with open('restaurants.sql', 'r') as f:
        for line in f:
            if line.strip().startswith("INSERT INTO restaurants VALUES"):
                match = re.search(r"\((.*)\);", line)
                if match:
                    restaurant_data.append([p.strip().strip("'") for p in match.group(1).split(',')])
    restaurants_df = pd.DataFrame(restaurant_data, columns=['restaurant_id', 'restaurant_name', 'cuisine', 'rating'])
    restaurants_df['restaurant_id'] = pd.to_numeric(restaurants_df['restaurant_id'])
    merged = pd.merge(orders_df, users_df, on='user_id', how='left')
    df = pd.merge(merged, restaurants_df, on='restaurant_id', how='left', suffixes=('_order', '_details'))
    df['order_date'] = pd.to_datetime(df['order_date'], dayfirst=True)

# Extract quarter
df['quarter'] = df['order_date'].dt.quarter

# Group by quarter and sum revenue
quarterly_revenue = df.groupby('quarter')['total_amount'].sum().sort_values(ascending=False)

print("Total Revenue by Quarter:")
print(quarterly_revenue)
print(f"\nHighest Revenue Quarter: Q{quarterly_revenue.idxmax()}")